# String matching

Given a `txt` string and a `pat` string, return a list of all the start indices for matches to `pat` in `txt`.

For example:

```
txt = "AABAACAADAABAABA"
pat = "AABA"
# 0  1  2  3  4  5  6  7  8  9  10  11  12  13  14  15
# A  A  B  A  A  C  A  A  D  A  A   B   A   A   B   A
# |--------|
#                            |----------|
#                                       |----------|

assert search(txt, pat) == [0,9,12]
```

## Naive string matching

In [140]:
def naive_search(txt, pat):
    n = len(txt)
    m = len(pat)
    matches = []
    for i in range(0, n - m + 1):
        is_match = True
        for j in range(len(pat)):
            if i+j >= len(txt) or pat[j] != txt[i+j]:
                is_match = False
                break
        if is_match:
            matches.append(i)
    return matches

assert naive_search("AABAACAADAABAABA", "AABA") == [0,9,12]

This solution has O((n-m+1)m) worst case time complexity, where n is the length of `txt` and m is the length of `pat`.

## Rabin-Karp algorithm

The idea of the Rabin-Harp algorithm is to precompute the hash value for `pat` and then compute a rolling hash value for each window of text the same length as `pat` in `txt`. The next hash value in the rolling hash can be efficiently computed by modifying the last hash value rather than computing it from scratch each step.

The implementation below is based on the following implementations:

1. http://web.archive.org/web/20230124203901/https://github.com/hansrajdas/algorithms/blob/master/Level-3/rabin_karp.py
2. http://web.archive.org/web/20221208004703/http://www-igm.univ-mlv.fr/~lecroq/string/node5.html
3. http://web.archive.org/web/20230124001119/https://algs4.cs.princeton.edu/53substring/RabinKarp.java.html

In [47]:
# Use the value of INT_MAX in C as the large
# prime number following implementation #2.
# 2**31 - 1 = 2147483647, which is prime.
q = 2**31 - 1

# The number of bits in the extended ASCII
# character set.
R = 256

def _hash(key, m):
    """Compute hash for key[:m]
    
    In particular:
    1) convert each character in key[:m] to its ASCII value
    2) convert those ASCII values to an integer in base-R,
       where R is the number of bits in the ASCII character set.
    3) take the integer modulo q to keep the hash value small and avoid
       integer overflow (not typically a problem in Python, but
       we leave it for educational purposes).
    """
    # Use Horner's method to evaluate the polynomial that
    # corresponds to the hash value in linear time.
    h = 0
    for j in range(m):
        h = (R*h + ord(key[j])) % q
    return h
        
def _check(txt, pat, i):
    for j in range(len(pat)):
        if pat[j] != txt[i + j]:
            return False
    return True

class RabinKarp:
    
    def __init__(self, pat):
        self.pat = pat
        m = len(pat)
            
        # precompute R**(m-1) % q for use in removing leading digit
        RM = 1
        for i in range(1, m):
            RM = (R * RM) % q
        self.RM = RM
        self.patHash = _hash(pat, m)

    def search(self, txt):
        n = len(txt)
        m = len(self.pat)
        
        matches = []
        if n < m:
            return matches

        txtHash = _hash(txt, m)

        if (self.patHash == txtHash) and _check(txt, self.pat, 0):
            matches.append(0)
        
        for i in range(m, n):
            # Compute the rolling hash.
            txtHash = (txtHash + q - self.RM*ord(txt[i-m]) % q) % q
            txtHash = (txtHash*R + ord(txt[i])) % q
            
            offset = i - m + 1
            if (self.patHash == txtHash) and _check(txt, self.pat, offset):
                matches.append(offset)
        
        return matches

assert RabinKarp("AABA").search("AABAACAADAABAABA") == [0,9,12]
assert RabinKarp("TEST").search("THIS IS A TEST TEXT") == [10]
assert RabinKarp("AAAA").search("AAAAABAAABA") == [0, 1]
assert RabinKarp("ABABCABAB").search("ABABDABACDABABCABAB") == [10]

This solution has O(m) worse case time complexity for preprocessing and O((n-m+1)m) worst case time complexity for the search, where n is the length of `txt` and m is the length of `pat`. Under reasonable assumptions, the expected time complexity for the matching is O(n) (see section 32.2 of CLRS for details).

## Finite automata

This implementation is based on section 32.3 of CLRS.

In [109]:
def compute_transition_function(pat, vocab):
    m = len(pat)
    v = len(vocab)
    delta = [[None for _ in range(v)] for _ in range(m + 1)]
    for q in range(m + 1):
        for i, ch in enumerate(vocab):
            # delta[q][i] gives the next state if
            # we're in state q and we have just read character ch,
            # In other words, it means that we have read pat[:q] and
            # then ch.
            #
            # ... pat[0] pat[1] ... pat[q] ch ...
            #                              ^
            #                          we're here
            #
            # The next state is the length of the longest prefix of
            # pat that matches a suffix of pat[:q] + ch.
            #
            # If q < m and ch == pat[q+1], then the longest
            # prefix is pat[:q+1], which gives us a next state of q+1,
            # which means we just move forward a state.
            # 
            # Otherwise, we move backward some number of states.
            # For example, suppose the pattern is "ababaca"
            # (Figure 32.7 in CLRS). And suppose we're here:
            #
            # ...ababac...
            #         ^
            # At this step, q = 5. Then, we read 'b'. The longest
            # prefix of pat that matches a suffix of "ababab" is
            # "abab". The length of that prefix is 4, so we move
            # back to state 4. This is moving backward the least
            # number of states possible.
            
            # CLRS uses min(m + 1, q + 2), but
            # min(m, q + 1) gives the same transition matrix
            # as Figure 32.7b. CLRS also states in the paragraph
            # below the algorithm that "The code starts with the
            # largest conceivable value of k, which is min(m, q+1)."
            k = min(m, q + 1) 
            while k > 0 and (not (pat[:q] + ch).endswith(pat[:k])):
                k -=1
            delta[q][i] = k
    return delta

vocab = ["a", "b", "c"]
pat = "ababaca"
delta = compute_transition_function(pat, vocab)
expected = [
    [1, 0, 0],
    [1, 2, 0],
    [3, 0, 0],
    [1, 4, 0],
    [5, 0, 0],
    [1, 4, 6],
    [7, 0, 0],
    [1, 2, 0]]
assert delta == expected

In [110]:
def finite_automaton_search(txt, delta, m, ch_to_idx):
    n = len(txt)
    q = 0
    matches = []
    for i in range(n):
        j = ch_to_idx[txt[i]]
        q = delta[q][j]
        if q == m:
            matches.append(i-m+1)
    return matches

txt = "AABAACAADAABAABA"
pat = "AABA"
m = len(pat)
vocab = ["A", "B", "C", "D"]
ch_to_idx = {ch: idx for idx, ch in enumerate(vocab)}
delta = compute_transition_function(pat, vocab)
m = len(pat)
assert finite_automaton_search(txt, delta, m, ch_to_idx) == [0, 9, 12]

The worst case time complexity of the preprocessing step in this solution is O(m^3 |vocab|) though it can be improved to O(m|vocab|) (see exercises 32.4-32.8 in CLRS). The worst case time complexity for the search is O(n).

## Knuth-Morris-Pratt algorithm

The implementation below is based on the [CLRS implementation](http://web.archive.org/web/20230125153025/https://algs4.cs.princeton.edu/53substring/KMP.java.html) in Java. It only finds the first occurrence of the pattern rather than all occurrences.

In [138]:
# The number of bits in the extended ASCII
# character set.
R = 256

class KMP:
    
    def __init__(self, pat):
        m = len(pat)
        dfa = [[0 for _ in range(m)] for _ in range(R)]
        dfa[ord(pat[0])][0] = 1
        x = 0
        for j in range(1, m):
            for c in range(R):
                dfa[c][j] = dfa[c][x]
            dfa[ord(pat[j])][j] = j + 1
            x = dfa[ord(pat[j])][x]
        self.dfa = dfa
        
    def search(self, txt):
        matches = []
        n = len(txt)
        m = len(self.dfa[0])
        i = j = 0
        while i < n and j < m:
            j = self.dfa[ord(txt[i])][j]
            i += 1
        if j == m:
            return i - m
        return n

assert KMP("AABA").search("AABAACAADAABAABA") == 0
assert KMP("TEST").search("THIS IS A TEST TEXT") == 10
assert KMP("AAAA").search("AAAAABAAABA") == 0
assert KMP("ABABCABAB").search("ABABDABACDABABCABAB") == 10

This solution has O(mR) worse case time complexity for preprocessing and O(n) worst case time complexity for the search. It also takes O(mR) space. An [improved version](http://web.archive.org/web/20230124195827/https://algs4.cs.princeton.edu/53substring/KMPplus.java.html) takes O(m) time for preprocessing and O(n) time for search, i.e. is independent of the vocab size/radix R, but "it's sufficiently more complicated that you should be prepared to study it carefully to really understand it" (Princeton KMP (Knuth Morris Pratt) Algorithm).

## Sources

* Chapter 32, "String matching", CLRS
* "[This website](http://www-igm.univ-mlv.fr/~lecroq/string/) is a great resource for exact string searching algorithms" (http://web.archive.org/web/20230124195819/https://algs4.cs.princeton.edu/53substring/)
* [43 5 Rabin Karp](https://www.youtube.com/watch?v=3KXsyZFHidk)
* [COMP526 4-3 §4.3 String matching with finite automata](https://www.youtube.com/watch?v=OJmM61Jnf1I)
* [Wikipedia page for Knuth–Morris–Pratt algorithm](http://web.archive.org/web/20230123223253/https://en.wikipedia.org/wiki/Knuth%E2%80%93Morris%E2%80%93Pratt_algorithm)
* [Princeton KMP (Knuth Morris Pratt) Algorithm](https://www.youtube.com/watch?v=iZ93Unvxwtw)
